# Wine Quality

## Notebook by [WenyiXu](https://github.com/xuwenyihust)

### Import libraries

In [3]:
%matplotlib inline

import numpy as np
import scipy as sp
import pandas as pd
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn.cross_validation

ImportError: No module named 'seaborn'